In [49]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

## Lecture des datas

In [50]:
df = pd.read_csv("all_data.csv", sep=";")
print(f"{len(df)} tickers")

1247 tickers


## Exclusions

### Tabac / Armes controversées

In [51]:
df2 = df.loc[(df['CWEAP_TIE']!=1) & (df['TOB_PRODUCER']!=1),:]
print(f"{len(df2)} tickers")

1237 tickers


Elimination de 10 entreprises

### Controverses 0/1

In [52]:
columns_controverse = ["E_CONTROVERSY", "S_CONTROVERSY", 'G_CONTROVERSY']

In [53]:
# Controverse E
for col in columns_controverse:
    print(f"NB {col} : {len(df.loc[df[col]<2,:])}")

NB E_CONTROVERSY : 28
NB S_CONTROVERSY : 149
NB G_CONTROVERSY : 35


In [54]:
df3 = df2.loc[~df[columns_controverse].lt(2).any(axis=1), :]
print(f"Elimination {len(df2)-len(df3)}")
print(f"Reste {len(df3)}")

Elimination 175
Reste 1062


### Note ESG < BB

In [55]:
print(f"NB : {len(df.loc[df["IVA_COMPANY_RATING"].isin(['B','CCC']),:])}")
df4 = df3.loc[~df["IVA_COMPANY_RATING"].isin(['B','CCC']),:]
print(f"Elimination {len(df3)-len(df4)}")
print(f"Reste {len(df4)}")

NB : 17
Elimination 11
Reste 1051


### Restrictions ODD

In [56]:
# odds = ["07","12","13"]
odds = [x for x in df.columns if x.startswith("SDG")]
for odd in odds:
    print(f"NB {odd} : {len(df.loc[df[odd]<=-2,:])}")

df5 = df4.loc[~df4[odds].le(-2).any(axis=1), :]
print(f"Elimination {len(df4)-len(df5)}")
print(f"Reste {len(df5)}")

NB SDG_01_NET_ALIGNMENT_SCORE : 28
NB SDG_02_NET_ALIGNMENT_SCORE : 7
NB SDG_03_NET_ALIGNMENT_SCORE : 48
NB SDG_04_NET_ALIGNMENT_SCORE : 16
NB SDG_05_NET_ALIGNMENT_SCORE : 1
NB SDG_06_NET_ALIGNMENT_SCORE : 14
NB SDG_07_NET_ALIGNMENT_SCORE : 83
NB SDG_08_NET_ALIGNMENT_SCORE : 6
NB SDG_09_NET_ALIGNMENT_SCORE : 37
NB SDG_10_NET_ALIGNMENT_SCORE : 9
NB SDG_11_NET_ALIGNMENT_SCORE : 51
NB SDG_12_NET_ALIGNMENT_SCORE : 106
NB SDG_13_NET_ALIGNMENT_SCORE : 83
NB SDG_14_NET_ALIGNMENT_SCORE : 64
NB SDG_15_NET_ALIGNMENT_SCORE : 66
NB SDG_16_NET_ALIGNMENT_SCORE : 28
NB SDG_17_NET_ALIGNMENT_SCORE : 70
Elimination 106
Reste 945


In [57]:
df5['EST_EU_TAXONOMY_MAX_REV'] = df5['EST_EU_TAXONOMY_MAX_REV'].fillna(0)
df5['Annualized return 10Y'] = df5['Annualized return 10Y'].fillna(0)
df5['Annualized return 10Y'] = df5['Annualized return 10Y'].fillna(0)
df5['INVEST_DURABLE'] = df5[['SDG_07_NET_ALIGNMENT_SCORE', 'SDG_12_NET_ALIGNMENT_SCORE', 'SDG_13_NET_ALIGNMENT_SCORE']].max(axis=1)

C:\Users\bapdu\AppData\Local\Temp\ipykernel_5924\3610090802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['EST_EU_TAXONOMY_MAX_REV'] = df5['EST_EU_TAXONOMY_MAX_REV'].fillna(0)
C:\Users\bapdu\AppData\Local\Temp\ipykernel_5924\3610090802.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Annualized return 10Y'] = df5['Annualized return 10Y'].fillna(0)
C:\Users\bapdu\AppData\Local\Temp\ipykernel_5924\3610090802.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [94]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# df6 = df5.loc[(df5['EST_EU_TAXONOMY_MAX_REV']>0) & (df5['ITR']>0),:] #Si Pas Taxo >0 trop de lignes => trop long a faire tourner

df6 = df5.loc[(df5['ITR']>0),:]
# Nombre de tickers
n = len(df6)
print(n)
perf_moy = df6['Annualized return 20Y'].values
taxonomie = df6['EST_EU_TAXONOMY_MAX_REV'].values
itr = df6['ITR'].values
durable = df6['INVEST_DURABLE']

secteurs_uniques = df6['gics_sector_name'].unique()  # Liste des secteurs uniques
m = len(secteurs_uniques)  # Nombre de secteurs

# Matrice binaire (1 si l'actif appartient au secteur, 0 sinon)
matrice_secteurs = np.array([[1 if df6.iloc[i]['gics_sector_name'] == secteur else 0 for secteur in secteurs_uniques] for i in range(n)])

# Fonction objectif (on minimise donc on prend -Perf_moy)
def objectif(w):
    return -np.dot(w, perf_moy)  # On minimise la valeur négative pour maximiser

# Contrainte : Taxonomie pondérée ≥ 10
def contrainte_taxonomie(w):
    return np.dot(w, taxonomie)-10.5

# Contrainte : ITR < 2
def contrainte_itr(w):
    return 1.95-np.dot(w, itr)

# Contrainte : Alignement environnemental pondéré ≥ 90%
def contrainte_alignement(w):
    filtres_durables = (durable >= 2).astype(int)  # 1 si durable > 2, sinon 0

    # Calcul de la somme des poids des entreprises ayant un score durable > 2
    return np.dot(w, filtres_durables) - 0.915  # Doit être ≥ 0

def contrainte_secteur(w):
    return 0.30 - np.dot(matrice_secteurs.T, w)  # Doit être ≥ 0 pour chaque secteur

# Contrainte : Somme des poids = 1
def contrainte_somme(w):
    return np.sum(w) - 1


# Définition des contraintes sous forme de dictionnaires
contraintes = [
    {'type': 'eq', 'fun': contrainte_somme},  # Somme des poids = 1
    {'type': 'ineq', 'fun': contrainte_taxonomie},  # Taxonomie pondérée ≥ 10
    {'type': 'ineq', 'fun': contrainte_itr},  # ITR < 2
    {'type': 'ineq', 'fun': contrainte_alignement}  # Investissement durable ≥ 90%
]

contraintes_secteurs = [{'type': 'ineq', 'fun': lambda w, j=j: contrainte_secteur(w)[j]} for j in range(m)]

contraintes = contraintes + contraintes_secteurs

# Bornes (chaque poids doit être positif et max 0.1)
bornes = [(0, 0.05) for _ in range(n)]

# Initialisation des poids (égalité répartie)
w0 = np.ones(n) / n

# Résolution avec SLSQP
resultat = minimize(objectif, w0, method='SLSQP', bounds=bornes, constraints=contraintes)

# Vérification des résultats
if resultat.success:
    w_opt = resultat.x
    df6['Poids'] = w_opt
else:
    print("L'optimisation a échoué :", resultat.message)

935


C:\Users\bapdu\AppData\Local\Temp\ipykernel_5924\922228086.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['Poids'] = w_opt


In [96]:
print(f"Return PTF : {(df6['Poids']*df6['Annualized return 20Y']).sum()}")
print(f"TAXO : {(df6['Poids']*df6['EST_EU_TAXONOMY_MAX_REV']).sum()}") 
print(f"ITR : {(df6['Poids']*df6['ITR']).sum()}") 
print(f"Poids : {(df6['Poids']).sum()}") 
filtres_durables = (df6['INVEST_DURABLE'] >= 2).astype(int)  # 1 si durable > 2, sinon 0.astype(int)  # 1 si durable > 2, sinon 0
print(f"Taxo : {(filtres_durables*df6['Poids']).sum()}") 

Return PTF : 0.275531139213899
TAXO : 10.499999999998572
ITR : 1.9499999999839717
Poids : 1.0000000000001996
Taxo : 0.9150000000000791


In [97]:
# Repartition sectorielle
round((df6.groupby('gics_sector_name')['Poids'].sum())*100,2)

gics_sector_name
Communication Services     8.50
Consumer Discretionary    15.94
Consumer Staples           0.00
Energy                     0.00
Financials                 7.55
Health Care               23.11
Industrials                5.00
Information Technology    30.00
Materials                  4.91
Real Estate                5.00
Utilities                  0.00
Name: Poids, dtype: float64

In [103]:
# Repartition geo
df_geo = df6.groupby('country')['Poids'].sum()
round(df_geo[df_geo>0.01]*100,2)

country
BRITAIN           5.00
DENMARK           5.00
FRANCE           15.00
GERMANY           5.00
NETHERLANDS       5.00
SWEDEN            9.91
UNITED STATES    55.09
Name: Poids, dtype: float64

In [104]:
df6[df6['Poids']>0.001]

,ticker,name,isin,country,currency,gics_sector_name,industry_group,region_en,Weight in MSCI World,Annualized return 20Y,...,FOOTPRINT_SCOPE_1,FOOTPRINT_SCOPE_2,FOOTPRINT_SCOPE_3,INTENSITY_SCOPE_1,INTENSITY_SCOPE_2,INTENSITY_SCOPE_3,EV,SALES,INVEST_DURABLE,Poids
1,NVDA US Equity,NVIDIA Corp,US67066G1040,UNITED STATES,USD,Information Technology,Semiconductors,North America,0.046132,0.396705,...,0.018164,0.224788,34.721009,0.131528,1.627755,251.425343,3.359642e+06,109406.527100,6.5,0.050000
11,V US Equity,Visa Inc,US92826C8394,UNITED STATES,USD,Financials,Diversified Finan Serv,North America,0.007654,0.208624,...,0.081778,0.532458,26.970073,0.297370,1.936173,98.071000,5.350970e+05,34700.923400,2.0,0.025452
14,MA US Equity,Mastercard Inc,US57636Q1040,UNITED STATES,USD,Financials,Diversified Finan Serv,North America,0.006298,0.289978,...,0.033665,0.459141,24.730839,0.135197,1.843861,99.316304,4.479303e+05,26302.422900,2.0,0.050000
19,NFLX US Equity,Netflix Inc,US64110L1061,UNITED STATES,USD,Communication Services,Internet,North America,0.005515,0.376162,...,0.353979,0.415922,115.939906,0.711690,0.836228,233.102162,3.089634e+05,36237.703140,1.0,0.035000
22,CRM US Equity,Salesforce Inc,US79466L3024,UNITED STATES,USD,Information Technology,Software,North America,0.004466,0.256363,...,0.062587,4.381089,59.867483,0.111356,7.794915,106.517341,2.710248e+05,35920.855100,3.5,0.050000
24,NOVOB DC Equity,Novo Nordisk A/S,DK0062498333,DENMARK,DKK,Health Care,Pharmaceuticals,Europe,0.004264,0.218411,...,0.706436,0.135853,67.557619,2.149638,0.413392,205.573224,4.682249e+05,36285.180300,2.0,0.050000
33,NOW US Equity,ServiceNow Inc,US81762P1021,UNITED STATES,USD,Information Technology,Software,North America,0.003177,0.232904,...,0.058149,1.315634,23.429482,0.240423,5.439600,96.871189,1.772127e+05,10107.177600,3.0,0.050000
60,BKNG US Equity,Booking Holdings Inc,US09857L1089,UNITED STATES,USD,Consumer Discretionary,Internet,North America,0.002389,0.318621,...,0.115734,0.004953,653.075464,0.169991,0.007276,959.241130,1.386878e+05,22265.926800,2.0,0.009367
113,VRTX US Equity,Vertex Pharmaceuticals Inc,US92532F1003,UNITED STATES,USD,Health Care,Biotechnology,North America,0.001512,0.211866,...,0.288912,0.542604,69.050536,0.747903,1.404633,178.750296,1.125805e+05,10255.346660,2.0,0.031080
155,CMG US Equity,Chipotle Mexican Grill Inc,US1696561059,UNITED STATES,USD,Consumer Discretionary,Retail,North America,0.001189,0.284092,...,7.758674,8.242878,109.326461,13.711046,14.566726,193.200545,7.951385e+04,10610.277240,2.0,0.050000
